# This is a jupyter notebook, running Python 3.6. 
We will use this notebook to import GL / TB demo data, perform some reconciliations, and then perform a few audit procedures.

## Load libraries

In [1]:
import pandas as pd

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

## Location of gl and tb files

In [3]:
tb = 'data/GLT0_0001_GL account master record transaction figures.xlsx'

In [4]:
gl = 'data/BSEG_0001_Accounting Document Segment.xlsx'

## Pull TB data into Dataframe

In [5]:
tb_df = pd.read_excel(tb, sheet_name=0)

### See what the first five records look like

In [ ]:
tb_df.head()

## Reshape the data to unpivot periods from columns to rows

In [ ]:
tb_column_renames = {
    'RACCT (Account Number)': 'GL_Account_Number',
    'RBUSA (Business Area)': 'Business_Unit_Code',
    'RYEAR (Fiscal Year)': 'Fiscal_Year',
    'RLDNR (Ledger)': 'Ledger'
}

In [ ]:
tb_df_melt = tb_df.melt(id_vars=tb_column_renames.keys(),
    value_vars=['TSL01 (Total transactions of the period in transaction currency)',
       'TSL02 (Total transactions of the period in transaction currency)',
       'TSL03 (Total transactions of the period in transaction currency)',
       'TSL04 (Total transactions of the period in transaction currency)',
       'TSL05 (Total transactions of the period in transaction currency)',
       'TSL06 (Total transactions of the period in transaction currency)',
       'TSL07 (Total transactions of the period in transaction currency)',
       'TSL08 (Total transactions of the period in transaction currency)',
       'TSL09 (Total transactions of the period in transaction currency)',
       'TSL10 (Total transactions of the period in transaction currency)',
       'TSL11 (Total transactions of the period in transaction currency)',
       'TSL12 (Total transactions of the period in transaction currency)'],
       var_name='Period', 
       value_name='Balance_As_Of_Date')

## Rename the period fields

In [ ]:
tb_df_melt['Period'] = tb_df_melt['Period'].map(lambda x: x[3:5])

In [ ]:
tb_df_melt = tb_df_melt.rename(columns=tb_column_renames)

In [ ]:
tb_df_melt.head()

In [ ]:
# Save file for import example
tb_df_melt.to_csv('data/Trial_Balance_YYYYMMDD_YYYYMMDD.csv', index=False)

# Pull data from gl into Dataframe

In [ ]:
gl_df = pd.read_excel(gl, sheet_name=0)

In [ ]:
#gl_df = gl_df[['BELNR (Accounting Document Number)', 'BUZEI (Number of Line Item Within Accounting Document)', 
#               'SGTXT (Item Text)', 'GSBER (Business Area)', 'AUGDT (Clearing Date)','GJAHR (Fiscal Year)',
#              'HKONT (General Ledger Account)', 'PSWBT (Amount for Updating in General Ledger)',
#              'SHKZG (Debit/Credit Indicator)', 'PSWSL (Update Currency for General Ledger Transaction Figures)']]

In [ ]:
gl_df.head()

In [ ]:
gl_column_rename_bseg = {
    'BELNR (Accounting Document Number)': 'Journal_ID',
    'BUZEI (Number of Line Item Within Accounting Document)': 'Journal_ID_Line_Number',
    'SGTXT (Item Text)': 'JE_Line_Description',
    'GSBER (Business Area)': 'Business_Unit_Code',
    'AUGDT (Clearing Date)': 'Effective_Date',
    'GJAHR (Fiscal Year)': 'Fiscal_Year',
    'HKONT (General Ledger Account)': 'GL_Account_Number',
    'PSWBT (Amount for Updating in General Ledger)': 'Amount',
    'SHKZG (Debit/Credit Indicator)': 'Amount_Credit_Debit_Indicator',
    'PSWSL (Update Currency for General Ledger Transaction Figures)': 'Amount_Currency'

}

gl_column_rename_bkpf = {
    'BKTXT (Document Header Text)': 'JE_Header_ Description',
    'BLART (Document Type)': 'Source',
    'USNAM (User name)': 'Entered_By',
    'BLDAT (Document Date in Document)' : 'Document_Date', 
    'CPUDT (Day On Which Accounting Document Was Entered)': 'Entered_Date',
    'CPUTM (Time of Entry)': 'Entered_Time',
    'BELNR (Accounting Document Number)': 'Journal_ID',
    'MONAT (Fiscal Period)': 'Period'
}

In [ ]:
# gl_df['Net'] = gl_df.apply(lambda x: x['PSWBT (Amount for Updating in General Ledger)']
#                            if x['SHKZG (Debit/Credit Indicator)'] == 'H'
#                           else (x['PSWBT (Amount for Updating in General Ledger)'] * -1),
#                           axis=1)

In [ ]:
gl_df = gl_df.rename(columns=gl_column_rename_bseg)

In [ ]:
cols = list(gl_column_rename_bseg.values())
gl_df_renamed = gl_df[cols]

In [ ]:
gl_df_renamed.head()

## Load file to pull other fields from

In [ ]:
xwalk = 'data/BKPF_0001_Accounting Document Header.TXT'

In [ ]:
xwalk_df = pd.read_csv(xwalk, sep='|', low_memory=False)

In [ ]:
xwalk_df = xwalk_df.rename(columns=gl_column_rename_bkpf)

In [ ]:
cols = list(gl_column_rename_bkpf.values())

In [ ]:
xwalk_final = xwalk_df[cols]

In [ ]:
xwalk_final.head()

In [ ]:
gl_df_final = pd.merge(gl_df_renamed, xwalk_final, on='Journal_ID', how='left')
gl_df_final.head()

In [ ]:
# Save the gl to csv
gl_df_final.to_csv('data/GL_Detail_YYYYMMDD_YYYYMMDD.csv', index=False)